In [20]:
from pathlib import Path
import json
from tqdm import tqdm
import pickle
import itertools

## Split

In [17]:
# Split all post texts
AD_PREV_TEXTS_FOLDER = Path("/home/deniskirbaba/Documents/influai-data/embeddings/ad_prev_texts")
AD_PREV_TEXTS_SPLITTER_FOLDER = Path("/home/deniskirbaba/Documents/influai-data/embeddings/ad_prev_texts_splitted")

for path in tqdm(AD_PREV_TEXTS_FOLDER.iterdir()):
    with open(path, "r") as f:
        texts = json.load(f)

    # Process each ad post
    for ad_post_id, prev_posts in tqdm(texts.items(), desc="Processing posts"):
        # If we have posts previous to the target ad post
        if prev_posts:
            doc = "\n===\n".join(itertools.chain.from_iterable(prev_posts[1]))
            with open(AD_PREV_TEXTS_SPLITTER_FOLDER / f"{ad_post_id}.txt", "w") as f:
                f.write(doc)

Processing posts: 100%|██████████| 10000/10000 [00:09<00:00, 1055.80it/s]
18it [04:00, 13.34s/it]


## Create not summarized ids list

In [18]:
AD_PREV_SUMMARIES_FOLDER = Path("/home/deniskirbaba/Documents/influai-data/embeddings/ad_prev_summaries")

len(list(iter(AD_PREV_TEXTS_SPLITTER_FOLDER.iterdir()))), len(list(iter(AD_PREV_SUMMARIES_FOLDER.iterdir())))

(177224, 0)

In [19]:
ad_prev_texts_stems = {path.stem for path in AD_PREV_TEXTS_SPLITTER_FOLDER.iterdir()}
ad_prev_summaries_stems = {path.stem for path in AD_PREV_SUMMARIES_FOLDER.iterdir()}

not_summarized = list(ad_prev_texts_stems - ad_prev_summaries_stems)
len(not_summarized)

177224

In [21]:
n_workers = 5
n_batch = (len(not_summarized) + n_workers) // n_workers

for i in range(0, len(not_summarized), n_batch):
    with open(f"/home/deniskirbaba/Documents/influai-data/embeddings/ad_prev_not_summarized/not_summarized_{i}.pkl", "wb") as f:
        pickle.dump(not_summarized[i : i + n_batch], f)